In [1]:
import config
import cx_Oracle

In [2]:
# connect to Oracle DB
connection = None
try:
    connection = cx_Oracle.connect(
        config.username,
        config.password,
        config.dsn,
        encoding = "UTF-8"
    )
    print("Connected")
    print(connection.version)
except cx_Oracle.Error as error:
    print(error)

cursor = connection.cursor()

Connected
12.2.0.1.0


In [ ]:
view_name = "V_HV_SEC_LN_1"


In [ ]:
# close connection
if connection:
    cursor.close()
    connection.close()
    print("Connection closed")